In [2]:
%pwd


'/blue/npadillacoreano/mcum/SocialMemEphys/diff_fam_social_memory_ephys/pilot2/only_subjects'

In [3]:
import os
os.chdir('/blue/npadillacoreano/mcum/SocialMemEphys/diff_fam_social_memory_ephys')



import numpy as np
import pandas as pd
import glob
from pathlib import Path
import matplotlib.pyplot as plt
from scipy import stats
import os
from scipy.stats import sem
from matplotlib.lines import Line2D
import sys
import pickle
import spike.spike_analysis.firing_rate_calculations as fr
import spike.spike_analysis.spike_collection as collection
import spike.spike_analysis.spike_recording as recording

import spike.spike_analysis.pca_trajectories as pca_trajectories
import pickle

In [12]:
only_subject_collection = collection.SpikeCollection(r'./data/only_subjects_spike_data')

loading  24_NFC_merged.rec
loading  13_NCF_merged.rec
loading  31_NFC_merged.rec
31_NFC_merged.rec has no good units
and will not be included in the collection
loading  22_FCN_merged.rec
loading  21_NCF_merged.rec
loading  41_FCN_merged.rec
loading  21_FCN_merged.rec
loading  23_NFC_merged.rec
loading  24_CNF_merged.rec
loading  44_FCN_merged.rec
loading  22_NCF_merged.rec
loading  32_NFC_merged.rec
loading  11_NCF_merged.rec
loading  23_CNF_merged.rec
loading  11_CNF_merged.rec
loading  12_CNF_merged.rec
Please assign event dictionaries to each recording
as recording.event_dict
event_dict = {event name(str): np.array[[start(ms), stop(ms)]...]
Please assign subjects to each recording as recording.subject


In [13]:
def pickle_this(thing_to_pickle, file_name):
    """
    Pickles things
    Args (2):   
        thing_to_pickle: anything you want to pickle
        file_name: str, filename that ends with .pkl 
    Returns:
        none
    """
    with open(file_name,'wb') as file:
        pickle.dump(thing_to_pickle, file)

pickle_this(only_subject_collection, 'only_subject_collection.pkl')